# Decision Tree

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("data/NBA_2014.csv",parse_dates=["Date"])
df.head()

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN


In [5]:
df["Homewin"] = df["HomePts"] > df["VisitorPts"]

In [6]:
df.head()

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,Homewin
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True


In [7]:
y_true = df["Homewin"].values
y_true

array([ True,  True,  True, ...,  True, False, False])

抽取特征

In [49]:
import collections
dic = collections.defaultdict(int)
dic

defaultdict(int, {})

In [50]:
df.head()

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,Homewin,HomeLastWin,VisitorLastWin
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,NaN,NaN
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,NaN,NaN
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True,NaN,NaN
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,NaN,NaN
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True,NaN,NaN


In [37]:
column_list = list(df.columns)

In [38]:
column_list.extend(["HomeLastWin","VisitorLastWin"])
column_list

['Date',
 'Score Type',
 'Visitor Team',
 'VisitorPts',
 'Home Team',
 'HomePts',
 'OT?',
 'Notes',
 'Homewin',
 'HomeLastWin',
 'VisitorLastWin']

In [44]:
df = df.reindex(columns=column_list)

In [46]:
df.head()

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,Homewin,HomeLastWin,VisitorLastWin
0,2013-10-29,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,NaN,NaN
1,2013-10-29,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,NaN,NaN
2,2013-10-29,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,True,NaN,NaN
3,2013-10-30,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,NaN,NaN
4,2013-10-30,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True,NaN,NaN


In [56]:
#是否赢得上一场比赛
won_last = collections.defaultdict(int)

for i, row in df.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    row['HomeLastWin'] = won_last[home_team]
    row['VisitorLastWin'] = won_last[visitor_team]
    df.ix[i] = row 
    
    #更新主客场特征值
    won_last[home_team] = row['Homewin']
    won_last[visitor_team] = not row['Homewin']


/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [58]:
df.iloc[20:25,:]

,Date,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,Homewin,HomeLastWin,VisitorLastWin
20,2013-11-01,Box Score,Milwaukee Bucks,105,Boston Celtics,98,NaN,NaN,False,False,False
21,2013-11-01,Box Score,Miami Heat,100,Brooklyn Nets,101,NaN,NaN,True,False,False
22,2013-11-01,Box Score,Cleveland Cavaliers,84,Charlotte Bobcats,90,NaN,NaN,True,False,True
23,2013-11-01,Box Score,Portland Trail Blazers,113,Denver Nuggets,98,NaN,NaN,False,False,False
24,2013-11-01,Box Score,Dallas Mavericks,105,Houston Rockets,113,NaN,NaN,True,True,True


sklearn 中用 CART 算法实现决策树

In [59]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
X_previouswins = df[["HomeLastWin","VisitorLastWin"]].values

In [60]:
X_previouswins

array([[0.0, 0.0],
       [0.0, 0.0],
       [0.0, 0.0],
       ...,
       [True, True],
       [True, False],
       [False, True]], dtype=object)

In [61]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf,X_previouswins,y_true,scoring='accuracy')
np.mean(scores)*100

/root/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


57.80487804878048

* 一般而言，什么球队水平更高？
* 两个球队相遇时，谁是赢家？

# 随机森林